Documentation : https://python.langchain.com/docs/integrations/tools/

### Custom tools

In [4]:
from langchain_core.tools import tool

In [1]:
# step 1 - create a function

def mutliply(a,b):
    """Multiply two numbers"""
    return a*b

In [2]:
# Step 2 : add type hints
def multiply(a : int, b : int) -> int:
    """Multiply two numbers"""
    return a*b

In [9]:
# Step 3 : add tool decorator

@tool 
def multiply(a : int, b : int) -> int:
    """Multiply two numbers"""
    return a*b

In [10]:
result = multiply.invoke({"a":3, "b":5})

In [11]:
print(result)

15


In [12]:
print(multiply.name)
print(multiply.description)
print(multiply.args)


multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [13]:
# what llm sees when you run a tool
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


### Method 2 : Using StructuredTool


In [14]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [16]:
class MultiplyInput(BaseModel):
    a : int = Field(required = True, description= "The first number to multiply")
    b : int = Field(required = True, description="The Second number to multiply")

In [17]:
def multiply_func(a : int , b : int):
    return a*b

In [18]:
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = 'multiply',
    description="Multiply two numbers",
    args_schema = MultiplyInput
)

In [19]:
result = multiply_tool.invoke({'a':3,'b':4})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)


12
multiply
Multiply two numbers
{'a': {'description': 'The first number to multiply', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The Second number to multiply', 'required': True, 'title': 'B', 'type': 'integer'}}


### Step 3 : Using BaseTool Class

In [20]:
from langchain.tools import BaseTool
from typing import Type

In [21]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a : int = Field(required = True, description = "The first number to add")
    b : int = Field(required = True, description = "The second number to add")

In [22]:
class MultiplyTool(BaseTool):
    name : str = 'multiply'
    description: str = "Multiply two numbers"
    
    args_schema: Type[BaseModel] = MultiplyInput
    
    def _run(self, a : int, b : int) -> int:
        return a*b

In [23]:
multiply_tool = MultiplyTool()

In [25]:
results = multiply_tool.invoke({'a': 4, 'b': 5})

print(results)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

20
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Toolkit


In [26]:
from langchain_core.tools import tool

# custom tool
@tool 
def add( a : int, b : int) -> int:
    """Add two Numbers"""
    return a + b

@tool
def multiply(a : int, b : int) -> int:
    """Multiply two numbers"""
    return a * b

In [29]:
class MathToolkit:
    def get_tools(self):
        return[add,multiply]

In [30]:
toolkit = MathToolkit()

tools = toolkit.get_tools()

for tool in tools :
    print(tool.name, " => ", tool.description)

add  =>  Add two Numbers
multiply  =>  Multiply two numbers
